In [8]:
import csv
import os

def add_image_info_to_csv(image_path, class_name="Test", key_terms="Test"):
    csv_file_name = 'Image_Keyterms.csv'
    file_exists = os.path.isfile(csv_file_name)
    with open(csv_file_name, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(['Path', 'Class', 'Keyterms'])
        writer.writerow([image_path, class_name, key_terms])

image_path = "(1).jpg"
add_image_info_to_csv(image_path)


In [12]:
import csv
import os
import ast

def add_image_info_to_csv(image_path, class_name, key_terms):
    csv_file_name = 'Image_Keyterms.csv'
    file_exists = os.path.isfile(csv_file_name)
    with open(csv_file_name, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(['Path', 'Class', 'Keyterms'])
        image_path = "'{}'".format(image_path)
        class_name = "'{}'".format(class_name)
        writer.writerow([image_path, class_name] + [key_terms])

image_path = "(1).jpg"
class_name = "Scenery"
key_terms = ["Sunset", "Beach", "Sea"]
add_image_info_to_csv(image_path, class_name, key_terms)
